In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
# %mpy -s {"/dev/cu.usbmodem_fs3_1"}
# %mpy -s {"/dev/cu.usbmodem1234561"}
# %mpy -s {"/dev/cu.usbmodem2101"}
# %mpy -s {"/dev/cu.usbmodem1101"}
%mpy -s {"/dev/cu.usbmodem101"}

In [ ]:
!mpremote reset

In [ ]:
import os
from contextlib import contextmanager
from pathlib import Path

@contextmanager
def cd(new_dir: str):
    original_dir = os.getcwd()

    try:
        os.chdir(new_dir)
        yield
    finally:
        os.chdir(original_dir)

# Create mp_libs dir in case it doesn't already exist
_ = !mpremote mkdir :/mp_libs || true

# Change directories to mp_libs root dir
notebooks_dir = Path(os.getcwd())
mp_libs_dir = notebooks_dir.parent

files = []
dirs = []
with cd(mp_libs_dir):
    # Get list of mp_libs files
    for path in Path(".").rglob("*"):
        if path.is_file() and path.suffix == ".py":
            files.append(path)
        if path.is_dir():
            dirs.append(path)

    # Create all sub-dirs
    for d in dirs:
        _ = !mpremote mkdir :mp_libs/"{d}" || true

    # Copy all mp_libs files over
    for f in files:
        base_dir = f"mp_libs/{f.parent if f.parent != Path('.') else ''}/"
        !mpremote cp "{f}" :"{base_dir}"



In [ ]:
mac = !mpremote exec 'import network; wlan = network.WLAN(network.STA_IF); wlan.active(True); print(wlan.config("mac"))'
print(f"{mac[0]=}")

In [ ]:
import tempfile


config_data = r"""
# Standard imports
from micropython import const

# Third party imports
from mp_libs import logging

config = {
    # Platform Configuration

    # App Configuration
    "device_name": "light_sensor",
    "light_sleep_sec": const(5),
    "deep_sleep_sec": const(120),
    "upload_rate_sec": const(120),
    "receive_rate_sec": const(120),
    "receive_window_sec": const(0.5),
    "time_sync_rate_sec": const(600),
    "send_discovery_rate_sec": const(600),
    "display_refresh_rate_sec": const(120),
    "ambient_pressure": const(1000),
    # "temp_offset_c": const(4.4),
    "temp_offset_c": const(0.5),
    "force_deep_sleep": True,
    "fake_sleep": True,
    "logging_level": logging.INFO,
    "log_to_fs": False,
    "log_to_buffer": False,
    "buffer_logging_level": logging.INFO,
    "debug": True,

    # Display Configuration
    "display_enable": False,

    # Transport Configuration
    # supported transports are: "espnow", "miniot", or "mqtt"
    "enable_network": True,
    "network_transport": "miniot",
    "network_prefix_id": "CO2",

    # Wifi configuration parameters
    "wifi_channel": const(8),

    # Mqtt configuration parameters
    "topics": {
        "pressure_topic": "homeassistant/aranet/pressure",
        "cmd_topic": "homeassistant/number/generic-device/cmd",
    },
    "keep_alive_sec": const(60),
    "keep_alive_margin_sec": const(20),
    "connect_retries": const(5),
    "recv_timeout_sec": const(10),
    "socket_timeout_sec": const(1),

    # Espnow configuration parameters
    # "epn_peer_mac": b'p\x04\x1d\xad|\xc0',
    # "epn_peer_mac": b'p\x04\x1d\xad}0',
    "epn_peer_mac": b"H\'\xe2\xc7n\xfc",
    "epn_channel": const(1),
    "epn_timeout_ms": 0
}
""".lstrip()

secrets_data=r"""
secrets = {
    'ssid': 'D&A_Home',
    'password': 'welivewiththeTrees7',
    'mqtt_broker': '192.168.0.19',
    'mqtt_port': 1883,
    'mqtt_username': 'hassio',
    'mqtt_password': 'hassio',
}
""".lstrip()

with tempfile.NamedTemporaryFile("w", delete=True) as tmp:
    tmp.write(config_data)
    tmp.flush()
    !mpremote fs cp {tmp.name} :config.py

with tempfile.NamedTemporaryFile("w", delete=True) as tmp:
    tmp.write(secrets_data)
    tmp.flush()
    !mpremote fs cp {tmp.name} :secrets.py
